In [42]:
from pymongo import MongoClient
import pymongo
import requests
import time
from bs4 import BeautifulSoup
import numpy as np
import datetime as dt
import pandas as pd
import math
from datetime import datetime
import time
from collections import Counter, defaultdict

In [2]:
client = MongoClient()
db = client['fencing']
fencers = db['fencers']
results = db['results']


In [3]:
page = '''https://askfred.net/Results/search.php?
ops%5Bfirst_name%5D=first_name&vals%5Bfirst_name%5D=&ops%
5Blast_name%5D=last_name&vals%5Blast_name%5D=&f%5Bweapon%5D=&ops%
5Bdate_1%5D=date_1_eq&vals%5Bdate_1%5D=&ops%5Bdate_2%
5D=date_2_eq&vals%5Bdate_2%5D=&f%5Bclub_id%5D=4160&f%
5Bcompetitor_division_id%5D=&ops%5Bplace%5D=place_eq&vals%
5Bplace%5D=&f%5Bevent_gender%5D=&f%5Bage_limit%5D=&f%
5Bevent_rating_limit%5D=&ops%5Bevent_rating%5D=event_rating_eq&vals%
5Bevent_rating%5D=&f%5Bis_team%5D=&ops%5Brating_earned%
5D=eq&vals%5Brating_earned_letter%5D=&vals%5Brating_earned_year%
5D=&f%5Btournament_name_contains%5D=&search_submit=
Search&page_id=2&page_id=3&page_id=4&page_id=3&page_id=2&page_id=1'''

In [4]:
test = requests.get(page)

In [5]:
test = requests.get(page)
souped = BeautifulSoup(test.content, 'html.parser')
evens = souped.select('tr.evenrow')
odds = souped.select('tr.oddrow')

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [144]:
evens[1]

<tr class="evenrow">
<td nowrap="">12/03/2017</td>
<td><a href="https://askfred.net/Results/index.php?tournament_id=38084&amp;FREDSID=p2ianqq97cl7fd0rcrae6v32b0">Five Weapon D Meet</a></td>
<td align="right" nowrap="">Div3 Senior Mixed Foil</td>
<td align="center" nowrap="">D1</td>
<td align="right" nowrap="">3rd of 18</td>
<td><div>Wilbert, Matthew</div></td>
<td nowrap="">
<div class="div_club"><a href="#" onclick="return false;">MOE</a></div>
<div class="div_club" style="display:none"><a href="#" onclick="return false;">Moe Fencing Club</a></div>
</td>
<td nowrap=""></td>
</tr>

In [6]:
for i in range(len(str(evens[0]).split('<'))):
    print(i , " : " , str(evens[0]).split('<')[i])

0  :  
1  :  tr class="evenrow">

2  :  td nowrap="">12/03/2017
3  :  /td>

4  :  td>
5  :  a href="https://askfred.net/Results/index.php?tournament_id=38084&amp;FREDSID=p2ianqq97cl7fd0rcrae6v32b0">Five Weapon D Meet
6  :  /a>
7  :  /td>

8  :  td align="right" nowrap="">Div3 Senior Mixed Foil
9  :  /td>

10  :  td align="center" nowrap="">D1
11  :  /td>

12  :  td align="right" nowrap="">1st of 18
13  :  /td>

14  :  td>
15  :  div>Tsai, Ryan
16  :  /div>
17  :  /td>

18  :  td nowrap="">

19  :  div class="div_club">
20  :  a href="#" onclick="return false;">MOE
21  :  /a>
22  :  /div>

23  :  div class="div_club" style="display:none">
24  :  a href="#" onclick="return false;">Moe Fencing Club
25  :  /a>
26  :  /div>

27  :  /td>

28  :  td nowrap="">D2017
29  :  /td>

30  :  /tr>


when split on '<':

2 = date
5 = URL / event name
8 = event
10 = event rating
12 = finish and competitors
15 = name
20 = club


In [7]:
date = str(evens[0]).split('<')[2][-10:]

In [8]:
url_tourney = str(evens[0]).split('<')[5]#.index('"')

url = url_tourney[8:]
url_end = url.index('"')
url = url[:url_end]

event_start = url_tourney.index('>')+1
event = url_tourney[event_start:]

In [9]:
event = str(evens[0]).split('<')[8][27:]


In [10]:
event_rating = str(evens[0]).split('<')[10][-2:]


In [11]:
place_size = str(evens[0]).split('<')[12]
ps_start = place_size.index('>')+1
ps_splits = place_size[ps_start:].split(' ')
size = ps_splits[2]
place = ps_splits[0][:-2]


In [12]:
name = str(evens[0]).split('<')[15][4:]

In [13]:
club = str(evens[0]).split('<')[20]
club_start = club.index('>') + 1
club = club[club_start:]
club

'MOE'

In [72]:
def extract_details(line):
    line = str(line).split('<')
    
    date = line[2][-10:]
    date = datetime.strptime(date, '%m/%d/%Y')
    event = line[8][27:]
    event_rating = line[10][-2:]
    name = line[15][4:]
    
    
    url_tourney = line[5] #gets line containing url and tournament name
    url = url_tourney[8:]
    url_end = url.index('"')
    event_start = url_tourney.index('>')+1 #prepping line for extract
    
    url = url[:url_end] #extracting url and event
    tourney = url_tourney[event_start:]
    
    
    place_size = line[12] #gets line w/place & size
    
    ps_start = place_size.index('>')+1 #prepping line for extract
    ps_splits = place_size[ps_start:].split(' ')
    
    size = ps_splits[2] #extracting
    place = ps_splits[0][:-2]
    
    club = line[20]
    club_start = club.index('>') + 1
    club = club[club_start:]
    
    new_rating = line[28]
    new_rating_start = new_rating.index('>') + 1
    new_rating = new_rating[new_rating_start:]
    
    weapon = 'Saber'
    if 'Epee' in event:
        weapon = 'Epee'
    elif 'Foil' in event:
        weapon = 'Foil'
        
    bonus, base = award_points(place, size)
    
    try:
        results.insert_one({
        'date':date,
        'tourney':tourney, 
        'event_rating':event_rating,
        'name':name, 
        'event':event, 
        'weapon':weapon, 
        'place':place, 
        'size':size, 
        'club':club, 
        'url':url,
        'new_rating': new_rating,
        'bonus':bonus,
        'base': base,
        'total': bonus + base   
        })
    except:
        return False
    
    #return date, tourney, event_rating, name, event, weapon, place, size, club, url

In [73]:
def scrape_page(num, club_id):
    
    page = '''https://askfred.net/Results/search.php?
    ops%5Bfirst_name%5D=first_name&vals%5Bfirst_name%5D=&ops%
    5Blast_name%5D=last_name&vals%5Blast_name%5D=&f%5Bweapon%5D=&ops%
    5Bdate_1%5D=date_1_eq&vals%5Bdate_1%5D=&ops%5Bdate_2%
    5D=date_2_eq&vals%5Bdate_2%5D=&f%5Bclub_id%5D='''+str(club_id)+'''&f%
    5Bcompetitor_division_id%5D=&ops%5Bplace%5D=place_eq&vals%
    5Bplace%5D=&f%5Bevent_gender%5D=&f%5Bage_limit%5D=&f%
    5Bevent_rating_limit%5D=&ops%5Bevent_rating%5D=event_rating_eq&vals%
    5Bevent_rating%5D=&f%5Bis_team%5D=&ops%5Brating_earned%
    5D=eq&vals%5Brating_earned_letter%5D=&vals%5Brating_earned_year%
    5D=&f%5Btournament_name_contains%5D=&search_submit=
    Search&page_id=2&page_id=3&page_id=4&page_id=3&page_id=2&page_id='''+str(num)
    
    raw = requests.get(page)
    souped = BeautifulSoup(raw.content, 'html.parser')
    evens = souped.select('tr.evenrow')
    odds = souped.select('tr.oddrow')
    
    for i in range(len(odds)):
        try:
            return extract_details(evens[i])
            return extract_details(odds[i])
        except:
            return False
    

In [68]:
def update_club(club_id):
    for i in range(1, 100):
        if scrape_page(i, club_id) == False:
            return "Updated"

In [74]:
scrape_page(1, 4160)

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


False

In [75]:
update_club(4160)

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


'Updated'

In [76]:
results.find().count()

220

In [49]:
results.delete_many({})

In [43]:
results.create_index([('name',pymongo.ASCENDING), ('date',pymongo.DESCENDING), ('event',pymongo.DESCENDING)], unique=True)

'name_1_date_-1_event_-1'

In [53]:
for result in results.find({}):
    print(result)

{'_id': ObjectId('5a26c968faec1130bc65810d'), 'date': datetime.datetime(2017, 12, 3, 0, 0), 'tourney': 'Five Weapon D Meet', 'event_rating': 'D1', 'name': 'Tsai, Ryan', 'event': 'Div3 Senior Mixed Foil', 'weapon': 'Foil', 'place': '1', 'size': '18', 'club': 'MOE', 'url': 'https://askfred.net/Results/index.php?tournament_id=38084&amp;FREDSID=i19dhdpmm60qdmc0tm7142baj1', 'bonus': 10, 'base': 18, 'total': 28}
{'_id': ObjectId('5a26c968faec1130bc65810e'), 'date': datetime.datetime(2017, 12, 3, 0, 0), 'tourney': 'Five Weapon D Meet', 'event_rating': 'D1', 'name': 'Wilbert, Gwendolyn', 'event': 'Div3 Senior Mixed Foil', 'weapon': 'Foil', 'place': '3', 'size': '18', 'club': 'MOE', 'url': 'https://askfred.net/Results/index.php?tournament_id=38084&amp;FREDSID=i19dhdpmm60qdmc0tm7142baj1', 'bonus': 6, 'base': 16, 'total': 22}
{'_id': ObjectId('5a26c968faec1130bc65810f'), 'date': datetime.datetime(2017, 12, 3, 0, 0), 'tourney': 'Five Weapon D Meet', 'event_rating': 'D1', 'name': 'Wilbert, Matthew'

In [43]:
extract_details(odds[5])

(datetime.datetime(2017, 9, 29, 0, 0),
 'Moe Found Friday Foil Open #114',
 'NR',
 'Tsai, Philip',
 'Mixed Foil',
 'Foil',
 '8',
 '22',
 'MOE',
 'https://askfred.net/Results/index.php?tournament_id=38657&amp;FREDSID=jr7so8uu6ugres8osbvr4515e1')

In [46]:
def award_points(place, size):
    place = int(place)
    size = int(size)
    
    bonus_max = int(math.log(size, 2)) + 1
    
    if math.log(place, 2) == int(math.log(place, 2)):
        bracket = int(math.log(place , 2))
    else:
        bracket = int(math.log(place , 2)) + 1
    
    bonus = (bonus_max - bracket)*2
    if bonus < 0:
        bonus = 0
    base = size-place + 1
    
    return bonus, base

In [183]:
def search_page_grab(url):
    raw = requests.get(url)
    souped = BeautifulSoup(raw.content, 'html.parser')
    evens = souped.select('tr.evenrow')
    odds = souped.select('tr.oddrow')
    return evens, odds
    

In [156]:
for entry in odds:
    date, tourney, event_rating, name, event, place, size, club = extract_details(entry)
    print(name)
    print(event)
    print(date)
    print(tourney)
    print(place, 'out of', size)
    #print('place', place)
    bonus, base = award_points(place, size)
    print('bonus', bonus)
    print('base', base)
    print('total', base+bonus)
    print()
    

Baldini, Andrea
Senior Mixed Foil
08/25/2017
Moe Future Scholarship Foil Open
1 out of 26
bonus 10
base 26
total 36

Karamete, Irem
Senior Mixed Foil
08/25/2017
Moe Future Scholarship Foil Open
3 out of 26
bonus 6
base 24
total 30

LaRosee, Matthew
Senior Mixed Foil
08/25/2017
Moe Future Scholarship Foil Open
5 out of 26
bonus 4
base 22
total 26

Banks, Alex
Senior Mixed Foil
08/25/2017
Moe Future Scholarship Foil Open
8 out of 26
bonus 4
base 19
total 23

Wilbert, Matthew
Senior Mixed Foil
08/25/2017
Moe Future Scholarship Foil Open
10 out of 26
bonus 2
base 17
total 19

Wilbert, Gwendolyn
Senior Mixed Foil
08/25/2017
Moe Future Scholarship Foil Open
15 out of 26
bonus 2
base 12
total 14

Miall, Steven
Senior Mixed Foil
08/25/2017
Moe Future Scholarship Foil Open
17 out of 26
bonus 0
base 10
total 10

Jenkins, Hannah
Senior Mixed Foil
08/25/2017
Moe Future Scholarship Foil Open
21 out of 26
bonus 0
base 6
total 6

Kang, Daniel
Senior Mixed Foil
08/25/2017
Moe Future Scholarship Foil O

In [180]:
url = 'https://askfred.net/Results/search.php?ops%5Bfirst_name%5D=first_name&vals%5Bfirst_name%5D=Elishua&ops%5Blast_name%5D=last_name&vals%5Blast_name%5D=Litle&f%5Bweapon%5D=&ops%5Bdate_1%5D=date_1_eq&vals%5Bdate_1%5D=&ops%5Bdate_2%5D=date_2_eq&vals%5Bdate_2%5D=&f%5Bclub_id%5D=&f%5Bcompetitor_division_id%5D=&ops%5Bplace%5D=place_eq&vals%5Bplace%5D=&f%5Bevent_gender%5D=&f%5Bage_limit%5D=&f%5Bevent_rating_limit%5D=&ops%5Bevent_rating%5D=event_rating_eq&vals%5Bevent_rating%5D=&f%5Bis_team%5D=&ops%5Brating_earned%5D=eq&vals%5Brating_earned_letter%5D=&vals%5Brating_earned_year%5D=&f%5Btournament_name_contains%5D=&FREDSID=sdbc2mfrb15po1162jttctuc67&search_submit=Search'


In [184]:
evens, odds = search_page_grab(url)
evens

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[<tr class="evenrow">
 <td nowrap="">08/25/2017</td>
 <td><a href="https://askfred.net/Results/index.php?tournament_id=37182&amp;FREDSID=dck42ocerunfcgmaeeg9oip1c5">Moe Future Scholarship Foil Open</a></td>
 <td align="right" nowrap="">Senior Mixed Foil</td>
 <td align="center" nowrap="">NR</td>
 <td align="right" nowrap="">3rd of 26</td>
 <td><div>Litle, Elishua</div></td>
 <td nowrap="">
 <div class="div_club"><a href="#" onclick="return false;">MOE</a></div>
 <div class="div_club" style="display:none"><a href="#" onclick="return false;">Moe Fencing Club</a></div>
 </td>
 <td nowrap=""></td>
 </tr>, <tr class="evenrow">
 <td nowrap="">06/17/2017</td>
 <td><a href="https://askfred.net/Results/index.php?tournament_id=36447&amp;FREDSID=dck42ocerunfcgmaeeg9oip1c5">NEUSFA 2017 Pomme De Terre - TEAM EVENTS</a></td>
 <td align="right" nowrap="">Senior Men's Team Foil</td>
 <td align="center" nowrap=""></td>
 <td align="right" nowrap="">1st of 8</td>
 <td><div>Moe 1</div></td>
 <td nowrap=""

In [92]:
type(results.find_one()['total']

int

In [139]:
def pull_club(club_set, weapon_set):

    if type(club_set) != list:
        club_set = [club_set]
        
    if type(weapon_set) != list:
        weapon_set = [weapon_set]

    filt = [
        {"$match": {
        "club": {'$in' : club_set}}},
        {"$match": {
        "weapon": {'$in' : weapon_set}}},

        {"$group": 
            {
                "_id": "$name", 
                "events": {"$sum": 1},
                "base" : {"$sum":{"$sum": "$base"}},
                "bonus" : {"$sum":{"$sum": "$bonus"}},
                "total" : {"$sum":{"$sum": "$total"}}       
            }}, 

        {"$sort": {"total": -1} }
        ]
    return results.aggregate(filt)
    
#     "total points"
# ]
    #{"$sort": SON([("count", -1), ("_id", -1)])}



In [353]:
def pull_fencer(fencer):

#     if type(club_set) != list:
#         club_set = [club_set]
        
#     if type(weapon_set) != list:
#         weapon_set = [weapon_set]


#'date': datetime.datetime(2017, 12, 3, 0, 0), 'tourney': 'Five Weapon D Meet', 'event_rating': 'D1', 'name': 'Tsai, Ryan', 'event': 'Div3 Senior Mixed Foil', 'weapon': 'Foil', 'place': '1', 'size': '18', 'club': 'MOE', 'url': 'https://askfred.net/Results/index.php?tournament_id=38084&amp;FREDSID=i19dhdpmm60qdmc0tm7142baj1', 'bonus': 10, 'base': 18, 'total': 28}
    last, first = fencer.split(',')
    firstLast = first[1:] + ' ' + last

    return firstLast, list(results.find({'name':fencer},
                         {'_id':0,
                         'date': 1,
                         'tourney': 1,
                         'event':1,
                         'place': 1,
                         'size': 1,
                          'place': 1,
                          'base': 1,
                          'bonus':1,
                          'total':1,
                          'url':1
                         } ))
#         print(row)

#     filt = [
#         {"$match": {
#         "name": {'$in' : club_set}}},
#         {"$match": {
#         "weapon": {'$in' : weapon_set}}},

#         {"$group": 
#             {
#                 "_id": "$name", 
#                 "events": {"$sum": 1},
#                 "base" : {"$sum":{"$sum": "$base"}},
#                 "bonus" : {"$sum":{"$sum": "$bonus"}},
#                 "total" : {"$sum":{"$sum": "$total"}}       
#             }}, 

#         {"$sort": {"total": -1} }
#         ]
#     return results.aggregate(filt)

In [354]:
pull_fencer('Tsai, Philip')

('Philip Tsai',
 [{'base': 93,
   'bonus': 4,
   'date': datetime.datetime(2017, 11, 24, 0, 0),
   'event': "Y14 Men's Foil",
   'place': '61',
   'size': '153',
   'total': 97,
   'tourney': 'Cobra Challenge Super Youth (SYC), Y8  and Cadet',
   'url': 'https://askfred.net/Results/index.php?tournament_id=37816&amp;FREDSID=i19dhdpmm60qdmc0tm7142baj1'},
  {'base': 8,
   'bonus': 2,
   'date': datetime.datetime(2017, 10, 20, 0, 0),
   'event': 'Mixed Foil',
   'place': '10',
   'size': '17',
   'total': 10,
   'tourney': 'Moe Found Friday Foil Open #115',
   'url': 'https://askfred.net/Results/index.php?tournament_id=38831&amp;FREDSID=k59c3mc48c7cm6kcgv3ubad0o2'},
  {'base': 36,
   'bonus': 6,
   'date': datetime.datetime(2017, 10, 14, 0, 0),
   'event': 'Div3 Senior Mixed Foil',
   'place': '5',
   'size': '40',
   'total': 42,
   'tourney': 'FVFA D &amp; Under Foil',
   'url': 'https://askfred.net/Results/index.php?tournament_id=37894&amp;FREDSID=53sm6nqt4sd4op9nh9ha8e0pv1'},
  {'base'

In [427]:
def create_fencers(list_of_names, club):
    
    if type(list_of_names) != list:
        list_of_names = list(list_of_names)

    tgtCols = ['Last Name',
                         'First Name',
                         'Birthdate',
                         'Birthdate verified',
                         'Club 1 Name',
                         'Club 1 Abbreviation',
                         'Club 1 ID#',
                         'Club 2 Name',
                         'Club 2 Abbreviation',
                         'Club 2 ID#',
                         'Member #',
                         'Member Type',
                         'Competitive',
                         'Expiration',
                         'Saber',
                         'Epee',
                         'Foil']
    
    
    membershipeoy = pd.read_csv('EOY-2017.csv')
    membershipcrnt = pd.read_csv('current_members.csv')
    membership = pd.merge(membershipeoy, membershipcrnt, on=tgtCols, how='outer')
    membership = membership[['Last Name',
                             'First Name',
                             'Birthdate',
                             'Birthdate verified',
                             'Club 1 Name',
                             'Club 1 Abbreviation',
                             'Club 1 ID#',
                             'Club 2 Name',
                             'Club 2 Abbreviation',
                             'Club 2 ID#',
                             'Member #',
                             'Member Type',
                             'Competitive',
                             'Expiration',
                             'Saber',
                             'Epee',
                             'Foil']]
    membership = membership.set_index(['Last Name', 'First Name'])
    membership = membership.dropna(thresh=11)
    
    for name in list_of_names:
        lastFirst = name.split(',')
        #print(name)
        #print(lastFirst)
        #print(lastFirst)
        first = lastFirst[1][1:]
        if first == 'Alex':
            first = 'Alexander'
        last = lastFirst[0]
        #print(last)
        
        try:
            truncated = membership.loc[last, first]
            
            #print(len(truncated))

            if len(truncated) > 1:
                truncated = truncated.iloc[0]
                byear = truncated[0]
            else:
                byear = truncated['Birthdate'][0]
            
            if type(byear) == str:
                byear = byear[-4:]
            
            byear = int(byear)

            foil = truncated['Foil'][0][:1]
            saber = truncated['Saber'][0][:1]
            epee = truncated['Epee'][0][:1]

            #print(byear)
            fencers.insert_one({
            'name': name,
            'byear': byear,
            'foil': foil,
            'saber': saber,
            'epee': epee   
            })
            
        except:
            print('Error on', name)
        
    
    

In [428]:
create_fencers(results.distinct('name'), 'MOE')

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:60: PerformanceWarning: indexing past lexsort depth may impact performance.


Error on Miao, Steven
Error on Wilbert, Matthew
Error on Yang, Meler
Error on Zhou, Daniel


In [465]:
time.

<module 'time' (built-in)>

In [464]:
for result in results.distinct('date'):
    print(result)

2017-12-03 00:00:00
2017-11-24 00:00:00
2017-11-25 00:00:00
2017-11-19 00:00:00
2017-11-04 00:00:00
2017-10-27 00:00:00
2017-10-22 00:00:00
2017-10-21 00:00:00
2017-10-20 00:00:00
2017-10-14 00:00:00
2017-10-07 00:00:00
2017-10-01 00:00:00
2017-09-29 00:00:00
2017-09-23 00:00:00
2017-09-24 00:00:00
2017-09-22 00:00:00
2017-09-16 00:00:00
2017-09-09 00:00:00
2017-09-08 00:00:00
2017-09-01 00:00:00
2017-08-26 00:00:00
2017-08-25 00:00:00
2017-06-23 00:00:00


In [451]:
categories = [
['A', 'B'],
['C', 'D'],
['E', 'U'],
]

def rating_groups(category, weapon, fencers_list):
    
    current = []
    
    for fencer in fencers_list:
        try:
            name = fencer['_id']
            record = fencers.find_one({'name': name})
            rating = record[weapon]
            
            if rating in category:
                current.append(fencer)
        except:
            pass
                
    return current
            

def age_groups(category, fencers_list):
    
    current = []
    
    for fencer in fencers_list:
        try:
            name = fencer['_id']
            record = fencers.find_one({'name': name})
            byear = record['byear']
            
            if byear >= category[0] and byear <= category[1]:
                print(byear, category[0], category[1])
                current.append(fencer)
        except:
            pass
                
    return current

In [452]:
print(age_groups([2003, 2006], pull_club('MOE', 'Foil')))

2005 2003 2006
2004 2003 2006
2003 2003 2006
2003 2003 2006
2003 2003 2006
2003 2003 2006
2003 2003 2006
2003 2003 2006
2005 2003 2006
2003 2003 2006
2003 2003 2006
2003 2003 2006
2003 2003 2006
2005 2003 2006
2004 2003 2006
[{'_id': 'Tsai, Max', 'events': 11, 'base': 319, 'bonus': 30, 'total': 349}, {'_id': 'Du, Hannah', 'events': 22, 'base': 281, 'bonus': 56, 'total': 337}, {'_id': 'Tsai, Philip', 'events': 8, 'base': 287, 'bonus': 30, 'total': 317}, {'_id': 'Sullivan, Jackson', 'events': 7, 'base': 143, 'bonus': 32, 'total': 175}, {'_id': 'Jenkins, Hannah', 'events': 8, 'base': 134, 'bonus': 12, 'total': 146}, {'_id': 'Yoon, Eugene', 'events': 8, 'base': 104, 'bonus': 18, 'total': 122}, {'_id': 'Antonio, Marcky', 'events': 5, 'base': 54, 'bonus': 6, 'total': 60}, {'_id': 'Gardiner, Sophia', 'events': 5, 'base': 35, 'bonus': 6, 'total': 41}, {'_id': 'McLean, Miles', 'events': 1, 'base': 16, 'bonus': 10, 'total': 26}, {'_id': 'Ma, Claire', 'events': 5, 'base': 18, 'bonus': 0, 'total':

1
1
1
1
1
1
1
2002.0
1
1
1
1
1
1
1
2003.0
1993.0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [339]:
fencers_list = pull_club('MOE', 'Foil')

for cat in categories:
    print(cat)
    print(rating_groups(cat, 'foil', pull_club('MOE', 'Foil')))
    print('\n')
    
# rating_groups(['C', 'B'], 'foil', fencers_list)

['A', 'B']
[{'_id': 'Gruber, Samuel', 'events': 7, 'base': 201, 'bonus': 42, 'total': 243}, {'_id': 'Karamete, Irem', 'events': 6, 'base': 157, 'bonus': 32, 'total': 189}, {'_id': 'Andrade, Nicholas', 'events': 5, 'base': 104, 'bonus': 40, 'total': 144}, {'_id': 'Litle, Elishua', 'events': 4, 'base': 87, 'bonus': 28, 'total': 115}, {'_id': 'Baldini, Andrea', 'events': 3, 'base': 75, 'bonus': 28, 'total': 103}, {'_id': 'Sapienza, Stefano', 'events': 3, 'base': 69, 'bonus': 18, 'total': 87}, {'_id': 'Nunley, Victoria', 'events': 2, 'base': 35, 'bonus': 6, 'total': 41}, {'_id': 'Banks, Alex', 'events': 1, 'base': 19, 'bonus': 4, 'total': 23}]


['C', 'D']
[{'_id': 'Cheng, Thomas', 'events': 8, 'base': 406, 'bonus': 44, 'total': 450}, {'_id': 'Molho, Sofia', 'events': 8, 'base': 171, 'bonus': 32, 'total': 203}, {'_id': 'Moo, Gareth', 'events': 5, 'base': 135, 'bonus': 16, 'total': 151}, {'_id': 'LeDonne, Renaud', 'events': 5, 'base': 90, 'bonus': 22, 'total': 112}, {'_id': 'Tsai, Ryan', 'e

In [355]:
ageGroups = [
    'JUNIOR',
     'CADET',
     'Y14',
      'Y12',
      'Y10']

ages = [
[1999, 2005],
[2002, 2005],
[2003, 2006],
[2005, 2008],
[2007, 2010]
]

In [336]:
for entry in fencers.find():
    print(entry['name'], entry['foil'])

Andrade, Nicholas A
Antonio, Marcky U
Baldini, Andrea A
Banks, Alex B
Bathon, Matthew U
Berger, Gregory E
Cheng, Thomas C
Danner-Doran, Griffin U
Du, Hannah U
Garber, Alejandro U
Gardiner, Luke U
Gardiner, Sophia U
Gilbert, Ian U
Gruber, Samuel A
Jenkins, Hannah E
Kang, Daniel U
Karamete, Irem A
Kelly, Katherine C
LaRosee, Matthew C
Lao, Sophia U
LeDonne, Raphael U
LeDonne, Renaud C
Litle, Elishua A
Ma, Claire U
McLean, Miles U
Miall, Steven U
Molho, Sofia C
Moo, Gareth C
Morell, Alfonso U
Nunley, Victoria B
Saldarriaga, Daniel U
Sapienza, Stefano A
Sullivan, Jackson E
Tecson, Michaela E
Thadhani, Elio U
Tsai, Chia-Lun Jack E
Tsai, Max U
Tsai, Philip E
Tsai, Ryan D
Vo, Antoine D
Wentworth, Marshall U
Wiest, Henry U
Wilbert, Gwendolyn D
Wong, Ethan U
Wong, Nathan D
Yang, Meler U
Yoon, Eugene U


In [166]:
membership = membership.set_index(['Last Name', 'First Name'])

In [391]:
tgtCols = ['Last Name',
                     'First Name',
                     'Birthdate',
                     'Birthdate verified',
                     'Club 1 Name',
                     'Club 1 Abbreviation',
                     'Club 1 ID#',
                     'Club 2 Name',
                     'Club 2 Abbreviation',
                     'Club 2 ID#',
                     'Member #',
                     'Member Type',
                     'Competitive',
                     'Expiration',
                     'Saber',
                     'Epee',
                     'Foil']

membershipeoy = pd.read_csv('EOY-2017.csv')
membershipcrnt = pd.read_csv('current_members.csv')
membership = pd.merge(membershipeoy, membershipcrnt, on=tgtCols, how='outer')
membership = membership[['Last Name',
                         'First Name',
                         'Birthdate',
                         'Birthdate verified',
                         'Club 1 Name',
                         'Club 1 Abbreviation',
                         'Club 1 ID#',
                         'Club 2 Name',
                         'Club 2 Abbreviation',
                         'Club 2 ID#',
                         'Member #',
                         'Member Type',
                         'Competitive',
                         'Expiration',
                         'Saber',
                         'Epee',
                         'Foil']]
membership = membership.set_index(['Last Name', 'First Name'])

In [432]:
#Saldarriaga
membership.loc['Yang'].head(200)#.iloc[0]['Birthdate'][-4:]

,Birthdate,Birthdate verified,Club 1 Name,Club 1 Abbreviation,Club 1 ID#,Club 2 Name,Club 2 Abbreviation,Club 2 ID#,Member #,Member Type,Competitive,Expiration,Saber,Epee,Foil
First Name,,,,,,,,,,,,,,,
Aaron,1/01/2006,No,NaN,NaN,NaN,Fairfax Fencers LLC,Fairfax Fencers Club,100025152.0,100212253,Non-Competitive,Non-Competitive,7/31/2017,U,U,U
Adam,1/01/2002,Yes,Olympia Fencing Center,Olympia F. C.,100025378.0,NaN,NaN,NaN,100133297,Competitive,Competitive,7/31/2017,U,U,C17
Alan,1/01/2007,No,NaN,NaN,NaN,Eagle Fencing Club,EFCA,100123604.0,100205781,Competitive,Competitive,7/31/2017,U,U,U
Alex,1/01/2002,No,Hills Fencing Institute (H3),H3,100093885.0,Half Hollow Hills Fencing Club,HALFHALLOW,100123614.0,100212221,Competitive,Competitive,7/31/2017,U,U,U
Andrew,1/01/2003,No,Salle Mauro Fencing Academy,SALLEMAURO,100033791.0,NaN,NaN,NaN,100181359,Competitive,Competitive,7/31/2017,U,U,E17
Ava,1/01/2000,No,Columbia High School Fencing Team,COLUMBH,100122910.0,NaN,NaN,NaN,100212941,High School Club Member,Non-Competitive,7/31/2017,U,U,U
Brandon,1/01/1998,No,Stevenson High School Fencing,Stevenson,100122976.0,NaN,NaN,NaN,100210772,High School Club Member,Non-Competitive,7/31/2017,U,U,U
Brandon,1/01/2000,No,Roswell High School Fencing Club,ROSWELL HS,100123255.0,NaN,NaN,NaN,100212804,High School Club Member,Non-Competitive,7/31/2017,U,U,U
Brenda,1/01/2000,No,Bergen Fencing Club,Bergen F.C.,100059051.0,NaN,NaN,NaN,100134240,Competitive,Competitive,7/31/2017,A16,U,U


In [392]:
print(len(membership))
membership = membership.dropna(thresh=11)
len(membership)

64754


42181

In [390]:
membership.head(30)

,,Birthdate,Birthdate verified,Club 1 Name,Club 1 Abbreviation,Club 1 ID#,Club 2 Name,Club 2 Abbreviation,Club 2 ID#,Member #,Member Type,Competitive,Expiration,Saber,Epee,Foil
Last Name,First Name,,,,,,,,,,,,,,,
Simmons,John,1/01/2007,Yes,Fencing Academy Of Boston,FAB,100025342.0,NaN,NaN,NaN,100170831,Competitive,Competitive,7/31/2017,U,U,U
Szilagyi,Eamon,1/01/-1,No,Silver City Fencing Club,Silver City,100128073.0,NaN,NaN,NaN,100223075,Competitive,Competitive,7/31/2018,U,U,U
Xu,Ethan,1/01/2008,No,International Fencers Alliance Of Boston,IFA,100128035.0,NaN,NaN,NaN,100221818,Competitive,Competitive,7/31/2018,U,U,U
',Shevek,1/01/1979,No,Golden Gate Fencing Center,Gold Gate,100029942.0,NaN,NaN,NaN,100133692,Competitive,Competitive,7/31/2017,U,D16,U
Aamer,Zain,1/01/2000,No,Stevenson High School Fencing,Stevenson,100122976.0,NaN,NaN,NaN,100210710,High School Club Member,Non-Competitive,7/31/2017,U,U,U
Aaron,Nicholas,1/01/1998,Yes,PDX Fencing,PDX,100099320.0,NaN,NaN,NaN,100088663,Competitive,Competitive,7/31/2017,D16,U,U
Abad,Mischa,1/01/2007,No,NaN,NaN,NaN,Tournament Fencers Club,Tournament Fencers,100025417.0,100212673,Supporting,Non-Competitive,7/31/2017,U,U,U
Abate,Derek,1/01/2008,No,NMFC/Duke City Fencing,Duke City,100067947.0,NaN,NaN,NaN,100217111,Non-Competitive,Non-Competitive,7/31/2017,U,U,U
Abayeva,Sasha,1/01/2002,Yes,Fencing Academy Of Westchester,FAW,100051910.0,NaN,NaN,NaN,100156258,Competitive,Competitive,7/31/2017,U,U,A17


In [320]:
# for row in pull_club('MOE', 'Epee'):
#     print(row['_id'],
#           'ev', row['events'],
#           'base', row['base'],
#           'bns', row['bonus'],
#           'tot',row['total'])

for entry in pull_club('MOE', 'Foil'):
    print(entry['_id'])

Cheng, Thomas
Miall, Steven
Tsai, Max
Du, Hannah
Tsai, Philip
Gruber, Samuel
Molho, Sofia
Karamete, Irem
Sullivan, Jackson
Moo, Gareth
Jenkins, Hannah
Andrade, Nicholas
Yoon, Eugene
Litle, Elishua
LeDonne, Renaud
Baldini, Andrea
Tsai, Ryan
Wilbert, Matthew
Sapienza, Stefano
LaRosee, Matthew
Wong, Ethan
Wilbert, Gwendolyn
Wiest, Henry
Tecson, Michaela
Wong, Nathan
Antonio, Marcky
Vo, Antoine
Kelly, Katherine
Nunley, Victoria
Gardiner, Sophia
Tsai, Chia-Lun Jack
Zhou, Daniel
McLean, Miles
Banks, Alex
Wentworth, Marshall
LeDonne, Raphael
Ma, Claire
Gilbert, Ian
Garber, Alejandro
Morell, Alfonso
Miao, Steven
Kang, Daniel
Saldarriaga, Daniel
Lao, Sophia
Yang, Meler
Gardiner, Luke
Berger, Gregory
Thadhani, Elio
Danner-Doran, Griffin


In [87]:
def dict_builder(url, results_dict):
    evens, odds = search_page_grab(url)
    full = evens + odds
    
    for entry in full:
        date, tourney, event_rating, name, event, place, size, club = extract_details(entry)
        bonus, base = award_points(place, size)
        
        event_dict = {
            'tournament': tourney,
            'event': event,
            'rating': event_rating, 
            'place': place,
            'size':size,
            'bonus':bonus,
            'base': base,
            'total': bonus+base}

        if name not in results_dict.keys():
            results_dict[name] = {}
            results_dict[name]['foil'] = {}
            results_dict[name]['saber'] = {}
            results_dict[name]['epee'] = {}
        if 'Foil' in event:
            weapon = 'foil'
        elif 'Epee' in event:
            weapon = 'epee'
        else:
            weapon = 'saber'
        results_dict[name][weapon][date] = event_dict
    

In [295]:
results_dict = {}
page = '''https://askfred.net/Results/search.php?
ops%5Bfirst_name%5D=first_name&vals%5Bfirst_name%5D=&ops%
5Blast_name%5D=last_name&vals%5Blast_name%5D=&f%5Bweapon%5D=&ops%
5Bdate_1%5D=date_1_eq&vals%5Bdate_1%5D=&ops%5Bdate_2%
5D=date_2_eq&vals%5Bdate_2%5D=&f%5Bclub_id%5D=4160&f%
5Bcompetitor_division_id%5D=&ops%5Bplace%5D=place_eq&vals%
5Bplace%5D=&f%5Bevent_gender%5D=&f%5Bage_limit%5D=&f%
5Bevent_rating_limit%5D=&ops%5Bevent_rating%5D=event_rating_eq&vals%
5Bevent_rating%5D=&f%5Bis_team%5D=&ops%5Brating_earned%
5D=eq&vals%5Brating_earned_letter%5D=&vals%5Brating_earned_year%
5D=&f%5Btournament_name_contains%5D=&search_submit=
Search&page_id=2&page_id=3&page_id=4&page_id=3&page_id=2&page_id='''
for i in range(5):
    dict_builder(page + str(i), results_dict)
    print(i)
    time.sleep(3)
    

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


0


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


1


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


2


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


3


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


4


In [146]:
for row in results.find({'name':'Tsai, Philip'}):
    print(row)

{'_id': ObjectId('5a26c968faec1130bc658117'), 'date': datetime.datetime(2017, 11, 24, 0, 0), 'tourney': 'Cobra Challenge Super Youth (SYC), Y8  and Cadet', 'event_rating': 'C3', 'name': 'Tsai, Philip', 'event': "Y14 Men's Foil", 'weapon': 'Foil', 'place': '61', 'size': '153', 'club': 'MOE', 'url': 'https://askfred.net/Results/index.php?tournament_id=37816&amp;FREDSID=i19dhdpmm60qdmc0tm7142baj1', 'bonus': 4, 'base': 93, 'total': 97}
{'_id': ObjectId('5a26cbf0faec1130bc65817e'), 'date': datetime.datetime(2017, 10, 20, 0, 0), 'tourney': 'Moe Found Friday Foil Open #115', 'event_rating': 'NR', 'name': 'Tsai, Philip', 'event': 'Mixed Foil', 'weapon': 'Foil', 'place': '10', 'size': '17', 'club': 'MOE', 'url': 'https://askfred.net/Results/index.php?tournament_id=38831&amp;FREDSID=k59c3mc48c7cm6kcgv3ubad0o2', 'bonus': 2, 'base': 8, 'total': 10}
{'_id': ObjectId('5a26cbf4faec1130bc658186'), 'date': datetime.datetime(2017, 10, 14, 0, 0), 'tourney': 'FVFA D &amp; Under Foil', 'event_rating': 'NR'

In [236]:
results_dict = {}
for entry in evens_odds:
    date, tourney, event_rating, name, event, place, size, club = extract_details(entry)
    bonus, base = award_points(place, size)
    event_dict = {
        'tournament': tourney,
        'rating': event_rating, 
        'event': event,
        'place': place, 
        'size':size,
        'bonus':bonus,
        'base': base,
        'total': bonus+base}
    
    if name not in results_dict.keys():
        results_dict[name] = {}
    results_dict[name][date] = event_dict

results_dict
    

{'Litle, Elishua': {'01/15/2016': {'base': 13,
   'bonus': 8,
   'event': 'Mixed Foil',
   'place': '1',
   'rating': 'NR',
   'size': '13',
   'total': 21,
   'tournament': 'Moe Found Friday Foil Open #87'},
  '02/03/2017': {'base': 18,
   'bonus': 8,
   'event': 'Mixed Foil',
   'place': '2',
   'rating': 'NR',
   'size': '19',
   'total': 26,
   'tournament': 'Moe Found Friday Foil Open #102'},
  '02/26/2016': {'base': 18,
   'bonus': 10,
   'event': 'Mixed Foil',
   'place': '1',
   'rating': 'NR',
   'size': '18',
   'total': 28,
   'tournament': 'Moe Found Friday Foil Open #90'},
  '03/11/2016': {'base': 17,
   'bonus': 10,
   'event': 'Mixed Foil',
   'place': '1',
   'rating': 'NR',
   'size': '17',
   'total': 27,
   'tournament': 'Moe Found Friday Foil Open #91'},
  '03/17/2017': {'base': 77,
   'bonus': 8,
   'event': "Div1A Senior Men's Foil",
   'place': '6',
   'rating': 'A3',
   'size': '82',
   'total': 85,
   'tournament': 'Boston Fencing Club D1A/D2/Bill Hall Vet ROC'

In [229]:
board = []
for key in results_dict:
    total = 0
    
    for date in results_dict[key]:
        total += results_dict[key][date]['total']
    board.append((total, key))
board = sorted(board, reverse=True)
for entry in board:
    print(entry[0], entry[1])
#total

350 Gruber, Samuel
250 Andrade, Nicholas
200 Cheng, Thomas
180 Sapienza, Stefano
179 Tsai, Philip
170 Molho, Sofia
167 Wong, Ethan
163 LaRosee, Matthew
131 Tsai, Max
124 Vo, Antoine
120 Sullivan, Jackson
111 Moo, Gareth
110 Baldini, Andrea
98 Tsai, Ryan
98 LeDonne, Renaud
84 Wong, Nathan
77 Nunley, Victoria
76 Jenkins, Hannah
73 Kelly, Katherine
68 Tecson, Michaela
63 McLean, Miles
61 Du, Hannah
60 Wilbert, Matthew
57 Litle, Elishua
56 Rea, Franklin T.
49 Banks, Alex
47 Yoon, Eugene
36 Tsai, Chia-Lun Jack
35 Jovanovic, Bojan
34 Domenicano, Ilaria
33 Wilbert, Gwendolyn
32 Wiest, Henry
30 Wencel, Clementine
30 Ma, Claire
30 Karamete, Irem
28 Zhou, Daniel
17 Miall, Steven
16 Moe 1
15 Romera, Julio
9 Gardiner, Sophia
8 Yang, Meler
7 Miao, Steven
7 Antoniv, Ivan
5 Morell, Alfonso
4 Wang, Zhao
4 Moe 1 Too
4 Kang, Daniel
2 Gardiner, Luke
2 Berger, Gregory
1 Tran, Arden


Tableau of 32

Top 16 + 2
Top 8 + 4
Top 4 + 6
Top 2 + 8
Top 1 + 10

In [301]:
for key in results_dict:
    for weapon in results_dict[key]:
        for date in results_dict[key][weapon]:
            print(key,
                  weapon,
                  date, 
                  results_dict[key][weapon][date]['place'],
                  results_dict[key][weapon][date]['size'])

Du, Hannah foil 08/26/2017 24 36
Du, Hannah foil 06/23/2017 16 24
Gruber, Samuel foil 08/25/2017 2 26
Gruber, Samuel foil 06/23/2017 3 24
Gruber, Samuel foil 06/17/2017 15 117
Litle, Elishua foil 08/25/2017 3 26
Moo, Gareth foil 08/25/2017 7 26
Moo, Gareth foil 06/23/2017 15 24
Moo, Gareth foil 06/17/2017 73 117
Nunley, Victoria foil 08/25/2017 9 26
Nunley, Victoria foil 06/23/2017 8 24
Nunley, Victoria foil 06/17/2017 23 56
Zhou, Daniel foil 08/25/2017 12 26
Yoon, Eugene foil 08/25/2017 16 26
Yoon, Eugene foil 06/17/2017 41 59
Wiest, Henry foil 08/25/2017 18 26
Wiest, Henry foil 06/23/2017 17 24
Wiest, Henry foil 06/17/2017 45 59
Gardiner, Sophia foil 08/25/2017 22 26
Gardiner, Sophia foil 06/23/2017 23 24
Ma, Claire foil 08/25/2017 24 26
Ma, Claire foil 06/23/2017 22 24
Ma, Claire foil 06/17/2017 32 40
Baldini, Andrea foil 08/25/2017 1 26
Karamete, Irem foil 08/25/2017 3 26
LaRosee, Matthew foil 08/25/2017 5 26
LaRosee, Matthew foil 06/23/2017 5 24
LaRosee, Matthew foil 06/17/2017 79

In [290]:
results_dict['Wilbert, Matthew']['08/25/2017']['event']

'Senior Mixed Foil'

In [159]:
membership = pd.read_csv('EOY-2017.csv')

In [323]:
for fencer in results_dict:
    #print(fencer)
    if ',' in fencer:
        brk = str(fencer).index(',')
        last = fencer[:brk]
        first = fencer[brk+2:]
        if ' ' in first:
            brk = first.index(' ')
            first = first[:brk]
        tdf = membership[membership['Last Name'] == last]
        tdf = tdf[tdf['First Name'] == first]
#         results_dict[fencer]['mf'] = tdf['Gender']
#         results_dict[fencer]['birth_year'] = tdf['Birthdate']
        blank = []
#         blank.append(tdf.loc[0]['Gender'])
#         blank.append(tdf.loc[0]['Birthdate'])
        print(fencer, blank)
        #str(tdf['Gender']), str(tdf['Birthdate']))
        print( )
        print(last)
        print(first)
        print()
        
# for fencer in results_dict:
#     print(results_dict[fencer])

KeyError: 'the label [0] is not in the [index]'

In [305]:
membership[membership['Last Name']=='Molho']

,Last Name,First Name,Middle Name,Suffix,Nickname,Gender,Birthdate,Birthdate verified,Division,Section,...,Expiration,Saber,Epee,Foil,US Citizen,Permanent Resident,Representing Country,Region #,Background Check Expires,SafeSport Expires
22847,Molho,Sofia,NaN,NaN,NaN,F,1/01/2002,Yes,New England,NaN,...,7/31/2018,U,U,C17,Yes,No,United States of America,3.0,NaN,NaN


In [455]:
testzip = zip(ages, ageGroups)

In [458]:
for zot in testzip:
    print(zot)

([1999, 2005], 'JUNIOR')
([2002, 2005], 'CADET')
([2003, 2006], 'Y14')
([2005, 2008], 'Y12')
([2007, 2010], 'Y10')


In [459]:
fencers.find_one({'name':'Cheng, Thomas'})

{'_id': ObjectId('5a29adb3faec1130bc65835e'),
 'byear': 1975,
 'epee': 'D',
 'foil': 'D',
 'name': 'Cheng, Thomas',
 'saber': 'U'}

In [460]:
membership.loc['Cheng', 'Thomas']

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  if __name__ == '__main__':


Birthdate Birthdate verified              Club 1 Name  \
Last Name First Name                                                          
Cheng     Thomas      1/01/1975                 No  Florida Fencing Academy   
          Thomas           2003                Yes     Moe Fencing Club LLC   
          Thomas           1975                 No  Florida Fencing Academy   

                     Club 1 Abbreviation   Club 1 ID#  \
Last Name First Name                                    
Cheng     Thomas                 FL F.A.  100030517.0   
          Thomas                  Moe FC  100089591.0   
          Thomas                 FL F.A.  100030517.0   

                                        Club 2 Name Club 2 Abbreviation  \
Last Name First Name                                                      
Cheng     Thomas      Winter Garden Fencing Academy                WGFA   
          Thomas                                NaN                 NaN   
          Thomas      Winter Garden Fencing Academy                WGFA   

                       Club 2 ID#   Member #  Member Type  Competitive  \
Last Name First Name                                                     
Cheng     Thomas      100084585.0  100199937  Competitive  Competitive   
          Thomas              NaN  100161804  Competitive  Competitive   
          Thomas      100084585.0  100199937  Competitive  Competitive   

                     Expiration Saber Epee Foil  
Last Name First Name                             
Cheng     Thomas      7/31/2017     U  D17  D17  
          Thomas      7/31/2018     U    U  C17  
          Thomas      7/31/2018     U  C17  D17